In [3]:
from PyQt5.QtWidgets import * 
from PyQt5.QtGui import * 
from PyQt5.QtCore import *
import os
import sys
import random
import pandas as pd
import joblib
from PandasModel import PandasModel
from PyQt5 import uic
from MLPClassifier import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy as np
np.set_printoptions(formatter={'float': '{: 3.4f}'.format})
import pyqtgraph as pg
from sklearn.metrics import accuracy_score
import itertools        
class UI(QWidget):
    def __init__(self):
        super().__init__()
        global aggdone
        aggdone=[0,0,0,0]
        uic.loadUi('ysagui.ui',self)
        self.coef_init = []
        self.intercept_init = []
        self.setWindowTitle("YSA Eğitim ve Test Programı")
        self.setWindowIcon(QIcon('iconycalc.png'))
        pd.options.display.float_format = '${:,.3f}'.format
        self.pathyuklebuton.clicked.connect(self.loadFile)
        self.agirlikeklebuton.clicked.connect(self.loadFile)
        self.agirlikislebuton.clicked.connect(self.loadFile)
        self.modelyuklebuton.clicked.connect(self.loadFile)
        self.modelyuklebuton_2.clicked.connect(self.loadFile)
        self.egitimmodelkaydetbuton.clicked.connect(self.saveFile)
        self.agirlikkaydetbuton.clicked.connect(self.saveFile)
        
        self.Datasetdurumlabel.setText("Dataset Yok")
        self.Datasetdurumlabel.setStyleSheet('color: black')
        aktivasyonstr = ['identity', 'logistic', 'tanh', 'relu']
        self.aktivasyoncomboBox.addItems(aktivasyonstr)
        lrlist = ["constant", "adaptive"]
        self.learningratecomboBox.addItems(lrlist)
        
        self.tabbar.setTabVisible(1,0)
        self.tabbar.setTabVisible(2,0)
        self.tabbar.setTabVisible(3,0)
        self.tabbar.setTabVisible(4,0)
        self.tabbar.setTabVisible(5,0)
        
        self.siniflandirbuton.setVisible(0)
        self.Traintestgecbuton.setVisible(0)
        self.modelegecbuton.setVisible(0)
        self.oransalspinbox.setVisible(0)
        self.nadetspinbox.setVisible(1)
        self.traintestayirbuton.setVisible(0)
        self.testegecbuton.setVisible(0)
        self.egitimmodelkaydetbuton.setVisible(0)
        
        self.siniflandirbuton.clicked.connect(self.siniflandirevent)
        self.Traintestgecbuton.clicked.connect(self.traintestgectim)
        self.traintestayirbuton.clicked.connect(self.traintestsplit)
        self.modelegecbuton.clicked.connect(self.modelgectim)
        self.agirlikeklemebuton.clicked.connect(self.agirlikekleevent)
        self.modelolusturbuton.clicked.connect(self.modelolusturevent)
        self.wmodelolusturbuton.clicked.connect(self.modelolusturevent)
        self.egitimbaslatbuton.clicked.connect(self.egitimevent)
        self.grafiktemizlebuton.clicked.connect(self.clearallgraph)
        self.testegecbuton.clicked.connect(self.egitimtest)
        self.secilenleritestetbuton.clicked.connect(self.testyap)
        self.temizlebuton.clicked.connect(self.temizle)
        [x.stateChanged.connect(self.checkboxevent) for x in self.findChildren(QCheckBox)]
        
        [x.setSizePolicy(Expanding, Expanding)for x in self.findChildren(QSizePolicy)]
    
    def temizle(self):
        self.sonuclardf=pd.DataFrame()
        modelsonuc=PandasModel(self.sonuclardf)
        self.sonuctablo.setModel(modelsonuc)           

    def siniflandirevent(self):
        
        self.siniflandirbuton.setVisible(0)
        
        global df,X,y
        
        for i in range(len(df.columns)-1,0,-1):
            if(df[df.columns[i]].dtypes==df[df.columns[i-1]].dtypes):

                continue
            else:
                ycol=range(i,len(df.columns))
                xcol=range(i)
                y=df[df.columns[ycol]].values
                X=df[df.columns[xcol]]
                X=X/X.max()
                self.updatepcikti("Girişler : %s \n Çıkışlar : %s \n Belirlendi.\n Girişler Normalize edildi.(0,1)"%(df.columns[xcol].values,df.columns[ycol].values))
                
                break
        for k in range(len(ycol)):        
            if(len(y.shape)!=1):
                if(len(np.unique(y))==2 ):
                    ycol = range(1,len(ycol)+1) 
                    y = pd.DataFrame(y,index=df.index,columns=ycol)
                    y = pd.from_dummies(y).astype('int64').values.reshape(len(df))
                    y = pd.DataFrame(y,index=df.index,columns=['Çıkış:Sınıf'])
                    self.updatepcikti("Çıkışların dummy coded olduğu belirlendi.\n Tek sütuna düşürüldü Çıkış:Sınıf olarak adlandırıldı.")
                    
                    break
                else:
                    np.append(x_col, ycol[0])
                    ycol.drop(ycol[0])
                    y = df[df.columns[ycol]]
                    X=df[df.columns[xcol]]
                    X=X/X.max()  
                    continue
                
            else:
                if(y.dtypes[0] != 'object' or 'string'):
                    y = df[df.columns[ycol]]
                    break
                else:
                    le = preprocessing.LabelEncoder()
                    le.fit(df[ycol])
                    y = le.transform(df[ycol])
                    y = pd.DataFrame(y,index=df.index,columns=['Çıkış:Sınıf'])
                    self.updatepcikti("Çıkışların label coded olduğu belirlendi.\n Tek sütuna düşürüldü Çıkış:Sınıf olarak adlandırıldı.")
                    
                    break
        
        dfc = X.copy()
        dfc = dfc.join(y)
        model = PandasModel(dfc)
        self.updatepcikti("Sınıflandırma Başarıyla Tamamlandı\n Giriş Size : %s \n Çıkış size : %s\n Sınıf Sayısı : %s"%(str(X.shape),str(y.shape),str(np.unique(y))))
        
        self.dfctablo.setModel(model)
        self.Traintestgecbuton.setVisible(1)
        [x.setChecked(1) for x in self.findChildren(QCheckBox)]
        
    def traintestgectim(self):
        
        self.tabbar.setTabVisible(1,1)
        self.tabbar.setCurrentIndex(1)
        
        
    def traintestsplit(self):
        global  X_train, X_test, y_train, y_test,traindex, testindex, dftest
        if(self.oransalcheckbox.isChecked() == True):
            testsize = self.oransalspinbox.value()
        elif(self.nadetcheckbox.isChecked() == True):
            testsize = (((len(np.unique(y)))/len(df))/len(np.unique(y)))*(self.nadetspinbox.value())
        if(self.randomcheckbox.isChecked() == True):
            homojen = None
        elif(self.homojencheckbox.isChecked() == True):
            homojen = y
        
        if (self.randomkatsayisabitcheckbox.isChecked() == True):
            randomkatsayı = self.randomkatsayispinbox.value()
        elif(self.randomkatsayirandomcheckbox.isChecked() == True):
            randomkatsayı = random.randint(0, 43)
        else:randomkatsayı = random.randint(0, 43)
        
        self.randomkatsayispinbox.setValue(randomkatsayı)
        
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testsize, random_state=randomkatsayı, stratify=homojen)
        traindex, testindex, y_traindex, y_testdex = train_test_split(df.index, y, test_size = testsize, random_state=randomkatsayı, stratify=homojen)
        
        
        dftrain = pd.DataFrame(X_train,index=traindex,columns=X.columns)
        dftrain.insert(len(X.columns),"Sınıf",y_train)
        dftest = pd.DataFrame(X_test,index=testindex,columns=X.columns)
        dftest.insert(len(X.columns),"Sınıf",y_test)
        modeltrain = PandasModel(dftrain)
        modeltest = PandasModel(dftest)
        
        self.homojenliktestdeger.setText("Homojenlik Score : "+str(len(np.unique(y_test))/len(np.unique(y)))+" / Örnek Sayısı : "+str(dftest.shape[0]))
        self.homojenliktestdeger.adjustSize()
        self.homojenliktraindeger.setText("Homojenlik Score : "+str(len(np.unique(y_train))/len(np.unique(y)))+" / Örnek Sayısı : "+str(dftrain.shape[0]))
        self.homojenliktraindeger.adjustSize()
        self.traintablo.setModel(modeltrain)
        
        self.testtablo.setModel(modeltest)
        
        self.testtablo_2.setModel(modeltest)

        self.modelegecbuton.setVisible(1)
        
        self.girisnorondeger.setText(str(len(X.columns)))
        self.cikisnorondeger.setText(str(len(np.unique(y))))
        
        self.updatepcikti('Train ve Test Verisi Ayrıldı\nTrain : %s \nTest : %s\nTrain Verisi Homojenlik : %s\nTest Verisi Homojenlik : %s\n'
               %(str(dftrain.shape),str(dftest.shape),str(len(np.unique(y_test))/len(np.unique(y))),str(len(np.unique(y_train))/len(np.unique(y)))))
        
        self.testindexlist = dftest.index.insert(0,'Hepsi').values.astype(str).tolist()
        self.testorneklercombobox.clear()
        self.testorneklercombobox.addItems(self.testindexlist)
        

   
    def checkboxevent(self, state):
        
        if self.sender() == self.oransalcheckbox:
            self.oransalspinbox.setVisible(self.oransalcheckbox.isChecked())
            self.nadetspinbox.setVisible(not(self.oransalcheckbox.isChecked()))
            self.nadetcheckbox.setChecked(not(self.oransalcheckbox.isChecked()))

        elif self.sender() == self.nadetcheckbox:
            self.oransalspinbox.setVisible(not(self.nadetcheckbox.isChecked()))
            self.nadetspinbox.setVisible(self.nadetcheckbox.isChecked())  
            self.oransalcheckbox.setChecked(not(self.nadetcheckbox.isChecked()))


        elif self.sender() == self.randomcheckbox:
            self.homojencheckbox.setChecked(not(self.randomcheckbox.isChecked()))
            
        elif self.sender() == self.homojencheckbox:
            
            if(self.homojencheckbox.isChecked()):
                self.oransalspinbox.setMinimum((((len(np.unique(y)))/len(df))/len(np.unique(y)))*(25))
                self.nadetspinbox.setMinimum(len(np.unique(y)))
            else:
                self.oransalspinbox.setMinimum(0.00001)
                self.nadetspinbox.setMinimum(0)
                
            self.randomcheckbox.setChecked(not(self.homojencheckbox.isChecked()))
            
        elif self.sender() == self.toleranscheckbox:
            self.validcheckbox.setChecked(not(self.toleranscheckbox.isChecked()))

        elif self.sender() == self.validcheckbox:
            self.toleranscheckbox.setChecked(not(self.validcheckbox.isChecked()))

        elif self.sender() == self.randomkatsayisabitcheckbox:
            self.randomkatsayirandomcheckbox.setChecked(not(self.randomkatsayisabitcheckbox.isChecked()))

        elif self.sender() == self.randomkatsayirandomcheckbox:
            self.randomkatsayisabitcheckbox.setChecked(not(self.randomkatsayirandomcheckbox.isChecked()))                

        elif self.sender() == self.agirliklarikendiolusturcheckbox:
            self.yuklenenagirlikkullancheckbox.setChecked(not(self.agirliklarikendiolusturcheckbox.isChecked()))
            self.agirlikeklemebuton.setVisible(not(self.agirliklarikendiolusturcheckbox.isChecked()))
            self.modelolusturbuton.setVisible(self.agirliklarikendiolusturcheckbox.isChecked())
            self.tabbar.setTabVisible(3,not(self.agirliklarikendiolusturcheckbox.isChecked()))
        elif self.sender() == self.yuklenenagirlikkullancheckbox:
            self.agirliklarikendiolusturcheckbox.setChecked(not(self.yuklenenagirlikkullancheckbox.isChecked()))
            self.agirlikeklemebuton.setVisible(self.yuklenenagirlikkullancheckbox.isChecked())
            self.modelolusturbuton.setVisible(not(self.yuklenenagirlikkullancheckbox.isChecked()))
            if self.yuklenenagirlikkullancheckbox.isChecked()==False:
                self.tabbar.setTabVisible(3,0)
        self.traintestayirbuton.setVisible((self.oransalcheckbox.isChecked() or self.nadetcheckbox.isChecked()) and (self.randomcheckbox.isChecked() or self.homojencheckbox.isChecked()))
        

    def modelgectim(self):
        
        self.tabbar.setTabVisible(2,1)
        self.tabbar.setCurrentIndex(2)
            
    def modelolusturevent(self):
        global ysa
        
        
        hiddenslayer = self.gizlinoronspinbox.value()
        learninrate = self.learningratespinbox.value()
        aktivasyon = self.aktivasyoncomboBox.currentText()
        lrstr = self.learningratecomboBox.currentText()
        maxiter = self.maxiterspinbox.value()
        karistir = self.karistircheckbox.isChecked()
        hatatolerans = self.toleransspinbox.value()
        momentumkatsayi = self.momentumspinbox.value()
        earlystoptolorval = self.validcheckbox.isChecked()
        validoran = self.validoranspinbox.value()
        niternochance = self.erkeniterspinbox.value()
        egitimkoru = False
        importw = self.yuklenenagirlikkullancheckbox.isChecked()
        
        if(importw==False):
            coef_initw=[]
            intercept_initw=[]
        else:  
            coef_initw=self.coef_initw
            intercept_initw=self.intercept_initw
        
        outputs = self.pcikti
        
        if self.sender()==self.wmodelolusturbuton:       
            if self.egitilmischeckbox.isChecked():
                outputs=[]
        
        ysa = MLPClassifier(hidden_layer_sizes=hiddenslayer, activation=aktivasyon, solver='sgd',
                learning_rate=lrstr, learning_rate_init=learninrate, max_iter=maxiter, shuffle=karistir,
                tol=hatatolerans, verbose=True,  momentum=momentumkatsayi, early_stopping=earlystoptolorval,
                validation_fraction=validoran,  n_iter_no_change=niternochance, random_state=None, warm_start=egitimkoru,
                importw=importw,coef_initw=coef_initw,intercept_initw=intercept_initw,outputs=outputs)
        
        if self.sender()==self.wmodelolusturbuton:
            if self.egitilmischeckbox.isChecked():
                ysa.partial_fit(X_test.values,y_test.values,np.unique(y))
                ysa.coefs_=self.coef_initw
                ysa.intercepts_=self.intercept_initw
                self.egitimtest()
        else:
            self.tabbar.setTabVisible(4,1)
            self.tabbar.setCurrentIndex(4)
            self.egitimbaslatbuton.setVisible(1)
        
        self.updatepcikti('Model Oluşturuldu')

    
    def egitimevent(self):
        global  X_train,y_train,X_test,y_test,traindex, testindex, dftest,modeller,modelindex,modelnamelist
        self.updatepcikti("Eğitim Başladı")
        self.egitimbaslatbuton.setVisible(0)
        ysa.fit(X_train.values,y_train.values.ravel())
        self.updatepcikti("Eğitim Bitti")
        color = (random.randint(0,256),random.randint(0,240),random.randint(0,240))
        if self.validcheckbox.isChecked():

            
            self.validgrafik.setTitle('Eğitim Valid Score')
            self.validgrafik.setLabel('bottom' , 'Iterasyon')
            self.validgrafik.setLabel('left','ValidScore')
            self.validgrafik.setBackground('w')
            pen = pg.mkPen(color=color)
            pen2 = pg.mkPen(color=color, width=10, style=Qt.DashLine)
            self.validgrafik.showGrid(x = True, y = True)
            self.validgrafik.addLegend()
            self.validgrafik.plot(range(ysa.n_iter_),ysa.validation_scores_,pen=pen)
            self.validgrafik.plot(x=[np.where(ysa.validation_scores_==ysa.best_validation_score_)[0][0]],y=[ysa.best_validation_score_],name = "Best Valid Skoru",symbol='+', symbolSize=15, symbolBrush=('b'),pen=pen2)
 
        self.lossgrafik.setTitle('Eğitim Loss Curve')
        self.lossgrafik.setLabel('left' ,'Loss')
        self.lossgrafik.setLabel('bottom', 'Iterasyon')
        self.lossgrafik.setBackground('w')
        pen = pg.mkPen(color=color)
        pen2 = pg.mkPen(color=color, width=10, style=Qt.DotLine)

        self.lossgrafik.showGrid(x = True, y = True)
        self.lossgrafik.addLegend()
        self.lossgrafik.plot(range(ysa.n_iter_),ysa.loss_curve_,pen=pen)

        if self.validcheckbox.isChecked():
            pointname = 'Best Valid Skoru için Best Loss'
        else:
            pointname = 'Best Loss'

        self.lossgrafik.plot(x=[np.where(ysa.loss_curve_==ysa.best_loss_)[0][0]],y=[ysa.best_loss_],name = pointname,symbol='+', symbolSize=15, symbolBrush=('b'),pen=pen2)
        self.testegecbuton.setVisible(1)
        self.egitimmodelkaydetbuton.setVisible(1)
        modeller.append(ysa)
        modelname=('egitimysa%d'%modelindex)
        modelnamelist.append(modelname)
        modelindex=modelindex+1
        self.modellercombobox.clear()
        self.modellercombobox.addItems(modelnamelist)
    def agirlikekleevent(self):
        self.tabbar.setTabVisible(3,1)
        self.tabbar.setCurrentIndex(3)
        
    def clearallgraph(self):
        self.validgrafik.clear()
        self.lossgrafik.clear()

    def updatepcikti(self,value):
        self.pcikti.append(value)
        # QApplication.processEvents()
    
    def egitimtest(self):

        self.tabbar.setTabVisible(5,1)
        self.tabbar.setCurrentIndex(5)

        self.modellercombobox.clear()
        self.modellercombobox.addItems(modelnamelist)
        
            
    def testyap(self):
        
        testettext = self.testorneklercombobox.currentText()
        testetindex=self.testorneklercombobox.currentIndex()
        
        testmodeladi = self.modellercombobox.currentText()
        testmodel = modeller[self.modellercombobox.currentIndex()]
        
        if testmodeladi != '':
            if testettext != 'Hepsi':
                if testettext != '':
                    probdf = pd.DataFrame(pd.DataFrame(testmodel.predict_proba(X_test.loc[int(testettext)][:].values.reshape(1, -1)),index=[int(testettext)]).max(axis=1),index=[int(testettext)],columns=['TahminYüzdesi'])
                    ypred = testmodel.predict(X_test.loc[int(testettext)][:].values.reshape(1, -1))
                    probdf.insert(0,'Model',modelnamelist[self.modellercombobox.currentIndex()])
                    probdf.insert(len(probdf.columns),'TahminDeğeri',ypred)
                    probdf.insert(len(probdf.columns),'GerçekDeğeri',y_test.loc[int(testettext)][:].values)
                    skor = accuracy_score(y_test.loc[int(testettext)][:].values.reshape(1, -1),ypred.reshape(1, -1))
                    probdf['TahminYüzdesi']=probdf['TahminYüzdesi']*100
                    probdf['TahminYüzdesi']=probdf['TahminYüzdesi'].map('%{:,.3f}'.format)
                    probdf.insert(len(probdf.columns),'Skor',skor.reshape(1, -1))
                    probdf.insert(0,'ÖrnekNo',int(testettext))
                    probdf.sort_values(by=['ÖrnekNo'])
                    probdf['Skor']=probdf['Skor'].replace(1,'Doğru')
                    probdf['Skor']=probdf['Skor'].replace(0,'Yanlış')
                    
                    self.sonuclardf=self.sonuclardf.append(probdf)
            
            elif testettext == 'Hepsi':
                for i in self.testindexlist:
                    if i!='Hepsi':
                        probdf = pd.DataFrame(pd.DataFrame(testmodel.predict_proba(X_test.loc[int(i)][:].values.reshape(1, -1)),index=[int(i)]).max(axis=1),index=[int(i)],columns=['TahminYüzdesi'])
                        ypred = testmodel.predict(X_test.loc[int(i)][:].values.reshape(1, -1))
                        probdf.insert(0,'Model',modelnamelist[self.modellercombobox.currentIndex()])
                        probdf.insert(len(probdf.columns),'TahminDeğeri',ypred)
                        probdf.insert(len(probdf.columns),'GerçekDeğeri',y_test.loc[int(i)][:].values)
                        skor = accuracy_score(y_test.loc[int(i)][:].values.reshape(1, -1),ypred.reshape(1, -1))
                        probdf['TahminYüzdesi']=probdf['TahminYüzdesi']*100
                        probdf['TahminYüzdesi']=probdf['TahminYüzdesi'].map('%{:,.3f}'.format)
                        probdf.insert(len(probdf.columns),'Skor',skor.reshape(1, -1))
                        probdf.insert(0,'ÖrnekNo',int(i))
                        probdf.sort_values(by=['ÖrnekNo'])
                        probdf['Skor']=probdf['Skor'].replace(1,'Doğru')
                        probdf['Skor']=probdf['Skor'].replace(0,'Yanlış')
                        self.sonuclardf=self.sonuclardf.append(probdf)
                    
            modelsonuc=PandasModel(self.sonuclardf)
            modelsonuc.sort(0,False)
            self.sonuctablo.setModel(modelsonuc)           

        
        

                
    def keyPressEvent(self, event):
            
        if event.key() == Qt.Key_Escape:
            Quitdlg = QMessageBox(self)
            Quitdlg.setWindowTitle("Uyarı!")
            Quitdlg.setText("%s kapatılsın mı?"%self.windowTitle())
            Quitdlg.setStandardButtons(QMessageBox.Yes | QMessageBox.No)
            Quitdlg.setIcon(QMessageBox.Question)
            click = Quitdlg.exec()
            if (click == QMessageBox.Yes):
                self.close()
                    
        if event.key() == Qt.Key_F11:
            base = self.geometry()
            if self.isMaximized():
                self.showNormal()
                
            else:
                self.showMaximized()
    
            
    def loadFile(self):
        
        global df,X,y,aggdone,modeller,modelindex,modelnamelist
        
        if self.sender()==self.pathyuklebuton:
            fileName, _ = QFileDialog.getOpenFileName(self, "Open File", "", "Excel Files (*.xlsx)");
            if (fileName!=""):
                _, datasetname = os.path.split(fileName)
                self.Datasetadilabel.setText(datasetname)
                self.Datasetadilabel.adjustSize()
                df = pd.read_excel(fileName)
                self.setWindowTitle("YSA Eğitim ve Test Programı : %s" % datasetname)
                modeller = []
                modelnamelist = []
                modelindex = 0
                self.modellercombobox.clear()
                self.modellercombobox.addItems(modelnamelist)
                self.updatepcikti('%s dataset yüklendi'%datasetname)

                self.Datasetdurumlabel.setText(" Dataset Yüklendi")
                self.Datasetdurumlabel.setStyleSheet('color: Green')
                self.siniflandirbuton.setVisible(1)
                
                [self.tabbar.setTabVisible(x,0) for x in range(1,6)]
                self.sonuclardf=pd.DataFrame()
            else:
                self.updatepcikti('Dataset Seçilmedi yada geçersiz yükleme başarısız')
            
            
        elif self.sender()==self.agirlikeklebuton:
            fileName, _ = QFileDialog.getOpenFileName(self, "Bias layer 1 yükle", "", "Excel Files (*.xlsx)");
            if (fileName!=""):
                _, datasetname = os.path.split(fileName)
                self.bias_init1 = pd.read_excel(fileName, header=None)
                if self.bias_init1.shape==(1,self.gizlinoronspinbox.value()):
                    self.updatepcikti('Bias layer 1 Ağırlık yüklendi')
                    aggdone[0]=1
                    modelbiasinit1=PandasModel(self.bias_init1)
                    self.biastablo1.setModel(modelbiasinit1)
                else:
                    self.updatepcikti('Bias layer 1 Ağırlık için shape uygun değil')
                    aggdone[0]=0
                fileName, _ = QFileDialog.getOpenFileName(self, "Bias layer 2 yükle", "", "Excel Files (*.xlsx)");
                if (fileName!=""):
                    _, datasetname = os.path.split(fileName)
                    self.bias_init2 = pd.read_excel(fileName, header=None)
                    if self.bias_init2.shape==(1,len(np.unique(y))):
                        self.updatepcikti('Bias layer 2 Ağırlık yüklendi')
                        aggdone[1]=1
                        modelbiasinit2=PandasModel(self.bias_init2)
                        self.biastablo2.setModel(modelbiasinit2)
                    else:
                        self.updatepcikti('Bias layer 2 Ağırlık için shape uygun değil')
                        aggdone[1]=0
                    fileName, _ = QFileDialog.getOpenFileName(self, "Coef layer 1 yükle", "", "Excel Files (*.xlsx)");
                    if (fileName!=""):
                        _, datasetname = os.path.split(fileName)
                        self.layer1_init = pd.read_excel(fileName, header=None)
                        if (self.layer1_init.shape==(X.shape[1],self.gizlinoronspinbox.value())):
                            self.updatepcikti('layer1 Ağırlık yüklendi')
                            aggdone[2]=1
                            modellayer1 = PandasModel(self.layer1_init)
                            self.layer1tablo.setModel(modellayer1)
                        else:
                            self.updatepcikti('layer1 Ağırlık için shape uygun değil')  
                            aggdone[2]=0
                        fileName, _ = QFileDialog.getOpenFileName(self, "Coef layer 2 yükle", "", "Excel Files (*.xlsx)");
                        if (fileName!=""):
                            _, datasetname = os.path.split(fileName)
                            self.layer2_init = pd.read_excel(fileName, header=None)
                            if (self.layer2_init.shape==(self.gizlinoronspinbox.value(),len(np.unique(y)))):
                                self.updatepcikti('layer2 Ağırlık yüklendi')
                                aggdone[3]=1
                                modellayer2=PandasModel(self.layer2_init)
                                self.layer2tablo.setModel(modellayer2)
                            else:
                                self.updatepcikti('layer2 Ağırlık için shape uygun değil')
                                aggdone[3]=0
                        else:
                            self.updatepcikti('2.Katman Ağırlık Seçilmedi yada geçersiz yükleme başarısız')
                                                    
                    else:self.updatepcikti('1.Katman Ağırlık Seçilmedi yada geçersiz yükleme başarısız')
                                           
                else:self.updatepcikti('Bias layer 2 Ağırlık Seçilmedi yada geçersiz yükleme başarısız')
                 
            else:self.updatepcikti('Bias layer 1 Ağırlık Seçilmedi yada geçersiz yükleme başarısız') 
            

            
        elif self.sender()==self.modelyuklebuton or self.sender()==self.modelyuklebuton_2:
            
            fileName, _ = QFileDialog.getOpenFileName(self, "Model Yükle", "", "Model File(*.ysa)");
            if (fileName!=""):
                _, modelname = os.path.split(fileName)
                modeller.append(joblib.load(modelname))
                modelnamelist.append(modelname)
                modelindex=modelindex+1
                self.modellercombobox.clear()
                self.updatepcikti('Model Yüklendi')
                self.modellercombobox.addItems(modelnamelist)
                self.egitimtest()
                

                    
        if aggdone==[1,1,1,1]:
            self.agirlikislebuton.setVisible(1)
            if self.sender()==self.agirlikislebuton:
                
                self.coef_initw=[self.layer1_init.values,self.layer2_init.values]
                
                self.intercept_initw=[self.bias_init1.values,self.bias_init2.values]
                
                self.updatepcikti('Bütün Ağırlıklar Kayıt Edildi.')
                self.wmodelolusturbuton.setVisible(1)
                
            else:
                self.updatepcikti('Bütün Ağırlıklar yüklendi')
                                      
            
        else:
            self.agirlikislebuton.setVisible(0)
            self.wmodelolusturbuton.setVisible(0)

    def saveFile(self):
        global modeller,modelnamelist
            
        if self.sender()==self.egitimmodelkaydetbuton:
                savefileName, _ = QFileDialog.getSaveFileName(self, "Model Kaydet", "ysamodel", "Model File(*.ysa)")
                if (savefileName!=""):
                    _, modelname = os.path.split(savefileName)
                    savemodel = modeller[self.modellercombobox.currentIndex()]
                    modelnamelist[self.modellercombobox.currentIndex()]=modelname
                    self.modellercombobox.clear()
                    self.modellercombobox.addItems(modelnamelist)
                    savemodel.outputs=[]
                    joblib.dump(savemodel, modelname)
                    self.updatepcikti('Eğitilmiş Model %s olarak kaydedildi'%modelname)
                    
                else:
                    self.updatepcikti('Kaydetme Başarısız')
                    
        elif self.sender()==self.agirlikkaydetbuton:
            savefileName, _ = QFileDialog.getSaveFileName(self, "Bias layer 1", "Biaslayer1", "Excel Files (*.xlsx)")
            if (savefileName!=""):
                savemodel = modeller[self.modellercombobox.currentIndex()]
                _, biasname = os.path.split(savefileName)           
                pd.DataFrame(savemodel.intercepts_[0].reshape(1,-1)).to_excel(biasname, header=False, index=False)
                savefileName, _ = QFileDialog.getSaveFileName(self, "Bias layer 2", "Biaslayer2", "Excel Files (*.xlsx)")
                self.updatepcikti('Biaslayer1 %s olarak Kaydedildi'%biasname)
                if (savefileName!=""):
                    _, biasname = os.path.split(savefileName)           
                    pd.DataFrame(savemodel.intercepts_[1].reshape(1,-1)).to_excel(biasname, header=False, index=False)
                    savefileName, _ = QFileDialog.getSaveFileName(self, "Coef layer 1", "Coeflayer1", "Excel Files (*.xlsx)")
                    self.updatepcikti('Biaslayer2 %s olarak Kaydedildi'%biasname)
                    if (savefileName!=""):
                        _, coefname = os.path.split(savefileName)           
                        pd.DataFrame(savemodel.coefs_[0]).to_excel(coefname, header=False, index=False)
                        savefileName, _ = QFileDialog.getSaveFileName(self, "Coef layer 2", "Coeflayer2", "Excel Files (*.xlsx)")
                        self.updatepcikti('layer1 %s olarak Kaydedildi'%coefname)
                        if (savefileName!=""):
                            _, coefname = os.path.split(savefileName)
                            pd.DataFrame(savemodel.coefs_[1]).to_excel(coefname, header=False, index=False)
                            self.updatepcikti('layer2 %s olarak Kaydedildi'%coefname)
                        else:
                            self.updatepcikti('Kaydetme Başarısız')
                    else:
                        self.updatepcikti('Kaydetme Başarısız')
                else:
                    self.updatepcikti('Kaydetme Başarısız')
            else:
                self.updatepcikti('Kaydetme Başarısız')


def main():
    if not QApplication.instance():

        app = QApplication(sys.argv)

    else:
        app = QApplication.instance() 
    w = UI()
    w.show()
    QApplication.setQuitOnLastWindowClosed(True)
    app.exec_()
    app.quit()
    
if __name__ == '__main__':
    main()